In [22]:
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from helper_sentiment import get_sentiment
from data_helper import select_first_n_occurences, main, main_2_pos_2_neg
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
col = "StoryBody"
df_bis = main_2_pos_2_neg()

30 rows


In [31]:
text = "Covid cases are increasing fast!"
get_sentiment([text, text, text], model, tokenizer, device, as_softmax=True)

tensor([[0.7236, 0.2287, 0.0477],
        [0.7236, 0.2287, 0.0477],
        [0.7236, 0.2287, 0.0477]])

## Second example

In [5]:
df_bis, _ = select_first_n_occurences(df_bis, n=2)

In [24]:
sentiment_df = pd.DataFrame(get_sentiment(df_bis[col].to_list(), model, tokenizer, device, as_softmax=True), index=df_bis.index, columns=['Negative', 'Neutral', 'Positive'])
df_res = pd.concat((df_bis, sentiment_df), axis=1)
display(df_res)
df_res = df_res.drop("StoryBody", axis=1).groupby('id_qis').mean()

,StoryBody,id_qis,Negative,Neutral,Positive
0,Excellent.,id3,0.019136,0.075585,0.905279
1,This is quite rare.,id2,0.051600,0.694163,0.254237
2,The CEO is completely crazy.,id1,0.885894,0.095818,0.018288
3,This is interesting.,id3,0.005564,0.092826,0.901610
4,This stock is overrated.,id1,0.890461,0.097445,0.012094
5,I love this firm.,id2,0.005037,0.015687,0.979276
6,I love this firm.,id2,0.005037,0.015687,0.979276
7,This is interesting.,id3,0.005564,0.092826,0.901610
8,i dislike it so much.,id5,0.924528,0.066234,0.009238
9,I would like to buy their items.,id2,0.011937,0.621672,0.366390


In [27]:
pd.DataFrame(cosine_similarity(df_res), index=df_res.index, columns=df_res.index)

id_qis,id1,id2,id3,id5
id_qis,,,,
id1,1.000000,0.095098,0.034071,0.999501
id2,0.095098,1.000000,0.914339,0.074829
id3,0.034071,0.914339,1.000000,0.025180
id5,0.999501,0.074829,0.025180,1.000000


In [29]:
for id_qi in df_res.index:
    print(f"{id_qi}:", " ".join(df_bis[df_bis['id_qis']==id_qi][col].to_list()))

id1: The CEO is completely crazy. This stock is overrated. This stock is overrated. The CEO is completely crazy. This is a scandal.
id2: This is quite rare. I love this firm. I love this firm. I would like to buy their items.
id3: Excellent. This is interesting. This is interesting. Excellent. Can't wait to see their new CEO! Can't wait to see their new CEO!
id5: i dislike it so much. i dislike it so much. It deserves to be ignored. It deserves to be ignored.
